In [1]:
# Included in Julia
using Random
using Statistics
using StatsBase
import Base.isless
using BSON: @load, @save
global won = false

"""STRUCTURE DEFINITION"""
abstract type Individual end

mutable struct FloatInd <: Individual
    genes::Array{Float64}
    fitness::Float64
end

function FloatInd(n::Int)
    FloatInd(randn(n), -Inf)
end

function float_init(population_size::Int, n::Int)
    [FloatInd(n) for i in 1:population_size]
end

function float_init(population_size::Int,n::Int,init_bson::Bool)
    @load "mymodel2.bson" ann
    
    array_genes = [FloatInd(n) for i in 1:population_size]
    layers = [ann.c1.w, ann.c1.b, ann.c2.w, ann.c2.b,ann.c3.w,ann.c3.b]
    
    for k in 1:(population_size%2)
        L = 1
        j = 1
        for i in eachindex(array_genes[k])
            if j > length(layers[L])
                L += 1
                j = 1
            end
            array_genes[k][i] = layers[L][j]
            j += 1
        end
    end
    array_genes
end
    
    

"""MUTATION AND EVALUATION"""
function isless(i1::Individual, i2::Individual)
    i1.fitness < i2.fitness
end

function evaluate!(population::Array{<:Individual},objective::Function)
    for i in population
        i.fitness = objective(i)
    end
end

function mutate(ind::FloatInd, mutation_rate)
    new_genes = copy(ind.genes)
    for i in eachindex(new_genes)
        if rand() < mutation_rate
            new_genes[i] = rand()
        end
    end
    FloatInd(new_genes, -Inf)
end

""" SELECTION """
function fp_select(population::Array{<:Individual})
    fits = [i.fitness for i in population]
    p = fits ./ sum(fits)
    sample(population, Weights(p))
end

function tournament_select(population::Array{<:Individual})
    tournament = sample(population, 3)
    sort!(tournament, rev=true)[1]
end
            
""" CROSSOVER """
function one_point_crossover(p1::Individual, p2::Individual)
    child = copy(p1.genes)
    n = rand(1:length(p2.genes))
    child[n:end] = copy(p2.genes[n:end])
    typeof(p1)(child, -Inf)
end

function uniform_crossover(p1::Individual, p2::Individual)
    child = copy(p1.genes)
    for i in eachindex(child)
        if rand() < 0.5
            child[i] = p2.genes[i]
        end
    end
    typeof(p1)(child, -Inf)
end

uniform_crossover (generic function with 1 method)

In [2]:
function step_ga(population::Array{<:Individual},objective::Function,select::Function,crossover::Function,mut_rate::Float64,num_elites::Int)
    evaluate!(population,objective)
    sort!(population, rev=true)
    max_fit = maximum([i.fitness for i in population])
    
    new_population = Array{Individual}(undef, 0)
    append!(new_population, population[1:num_elites])
    while length(new_population) < length(population)
        parent1 = select(population)
        parent2 = select(population)
        child1 = crossover(parent1, parent2)
        child1 = mutate(child1,mut_rate)
        push!(new_population, child1)
    end
    
    new_population, max_fit
end

function ga(N::Int,population_size::Int,objective::Function,sel::String,cross::String,mut_rate::Float64,num_elites::Int)
    population = float_init(population_size, N,true)
    if sel == "tournament_select"
        selection = tournament_select
    elseif sel =="fp_select"
        selection = fp_select
    end
    if cross =="one_point_crossover"
        crossover = one_point_crossover
    elseif cross == "uniform_crossover"
        crossover = uniform_crossover
    end
    
    i = 0
    fits = []
    while ~won && i<10
        i+=1
        start= time()
        population, max_fit = step_ga(population,objective,selection,crossover,mut_rate,num_elites)
        fits = [fits ; max_fit]
        println("generation = ",i,", fitness = ",max_fit,", time elapsed =",time()-start)
    end
    population,fits
end

ga (generic function with 1 method)

In [3]:
using PyCall
using Conda
using Flux
using Plots

retro = pyimport_conda("retro","gym")

"""NEURAL NETWORK DEFINITION"""
struct convol
    w ::AbstractArray{Float64}
    b ::AbstractArray{Float64}
end

struct my_CNN
    c1 :: convol
    c2 :: convol
    c3 :: convol
end

function my_CNN(f1::Int,c1_in::Int,c1_out::Int,f2::Int,c2_in::Int,c2_out::Int,c3_in::Int,c3_out::Int)
    c1 = convol(zeros(f1,f1,c1_in,c1_out),zeros(c1_out))
    c2 = convol(zeros(f2,f2,c2_in,c2_out),zeros(c2_out))
    c3 = convol(zeros(c3_out,c3_in),zeros(c3_out))
    my_CNN(c1,c2,c3)
end

function compute_cnn(ann,inputs)
    y = Flux.Conv(ann.c1.w,ann.c1.b,σ;stride=3)(inputs)
    y = Flux.MaxPool((2,2),stride=2)(y)
    y = Flux.Conv(ann.c2.w,ann.c2.b,σ;stride=3)(y)
    y = Flux.MeanPool((2,2),stride=4)(y)
    y = flatten(y)
    y = Flux.Dense(ann.c3.w,ann.c3.b,σ)(y)
    y = (y .> 0.5) .* 1 
end

compute_cnn (generic function with 1 method)

In [4]:
"""RELATED TO SONIC"""
function play_env(ann; render=false,train=true)
    env = retro.make("SonicTheHedgehog-Genesis","GreenHillZone.Act1")
    ob = env.reset()
    total_reward = 0.0
    done = false
    
    #inx, iny, inc = env.observation_space.shape
    #inx = floor(Int,inx/8)
    #iny = floor(Int,iny/8)
    
    max_fitness = 0
    fitness = 0
    counter = 0
    xpos = 0
    xpos_max = 0
    frame = 0
    while ~done
        if render
            frame+=1
            env.render()
        end
        
        ob = Flux.unsqueeze(ob,4)
        action = compute_cnn(ann,ob)
        #println("action = ",action)
        
        ob, reward, done, info = env.step(action)
    
        fitness += reward

        xpos = info["x"]
        xpos_end = info["screen_x_end"]
        
        if done 
            println("You won !")
            global won = true
        end

        if xpos > xpos_max
            fitness += 1
            xpos_max = xpos
        end

        if fitness > max_fitness
            max_fitness = fitness
            counter = 0
        else
            counter += 1
        end
        
        if train == true 
            if counter>350 && fitness <= 2000
                done = true
                println("max steps reached")
            end

            if counter>500 && fitness>2000
                done = true
                println("Good fitness but max steps reached")
            end
        end

    end
    
    env.close()
    fitness
end

function my_CNN(genes::Array{Float64})
    ann = my_CNN(12,3,4,8,4,1,12,12)
    layers = [ann.c1.w, ann.c1.b, ann.c2.w, ann.c2.b,ann.c3.w,ann.c3.b]
    L = 1
    j = 1
    for i in eachindex(genes)
        if j > length(layers[L])
            L += 1
            j = 1
        end
        layers[L][j] = genes[i]
        j += 1
    end
    ann
end

function objective(indiv::Individual)
    ann = my_CNN(indiv.genes)
    obj = play_env(ann;render=false)
end

objective (generic function with 1 method)

In [5]:
"""PARAMETERS DEFINITION"""
n = 12*12*3*4+4+8*8*4*1+1+12*12  #shouldn't be changed!
size_population = 10

selection = ["tournament_select","fp_select"][1] # choice
crossover = ["one_point_crossover","uniform_crossover"][2] #choice
mutation_rate = 1/n
num_elites = 2

2

In [6]:
best,fits = ga(n,size_population,objective,selection,crossover,mutation_rate,num_elites);

i = 1:length(fits)
plot(i,fits)
savefig("GA3.png")

max steps reached
max steps reached
max steps reached
max steps reached
max steps reached
max steps reached
max steps reached
max steps reached
max steps reached
max steps reached
generation = 1, fitness = 31.0, time elapsed =78.81200003623962
max steps reached
max steps reached
max steps reached
max steps reached
max steps reached
max steps reached
max steps reached
max steps reached
max steps reached
max steps reached
generation = 2, fitness = 177.0, time elapsed =76.30100011825562
max steps reached
max steps reached
max steps reached
max steps reached
max steps reached
max steps reached
max steps reached
max steps reached
max steps reached
max steps reached
generation = 3, fitness = 177.0, time elapsed =90.51799988746643
max steps reached
max steps reached
max steps reached
max steps reached
max steps reached
max steps reached
max steps reached
max steps reached
max steps reached
max steps reached
generation = 4, fitness = 489.0, time elapsed =93.35500001907349
max steps reached
max

In [7]:
max_fit = 0 
i_max=0
for (i,indiv) in enumerate(best) 
    if max_fit<indiv.fitness
        max_fit = indiv.fitness
        i_max=i
    end
end

best_indiv = best[i_max].genes
ann = my_CNN(best[1].genes)

using BSON: @save
@save "mymodel3.bson" ann

In [8]:
using BSON: @load
@load "mymodel-GA.bson" ann

In [ ]:
play_env(ann; render=true,train=false)